[PyTorch Original Impl.](https://github.com/pytorch/examples/blob/main/vae/main.py)

In [ ]:
!pip install -q wandb pytorch_lightning

In [ ]:
import argparse
import math
import torch
import torch.utils.data
import torchvision
import wandb
import pytorch_lightning as pl

from __future__ import print_function
from copy import deepcopy
from collections import OrderedDict
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping
from pytorch_lightning.tuner import Tuner
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR


pl.seed_everything(42)

# Data

In [3]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, config):
        super().__init__()
        self.batch_size = config.batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
        torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):
        if stage in ('fit', None):
            self.cifar10_train = torchvision.datasets.CIFAR10(
                root='./data', train=True, transform=self.transform)
            self.cifar10_val = torchvision.datasets.CIFAR10(
                root='./data', train=False, transform=self.transform)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2, # TODO: check if we need more workers
            pin_memory=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.cifar10_val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2, # TODO: check if we need more workers
            pin_memory=True
        )

# Model

In [4]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        if in_channels != out_channels:
            self.identity = nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=1
            )
        else:
            self.identity = nn.Identity()

        hidden_channels = out_channels // 4

        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=hidden_channels,
                kernel_size=3,
                padding=1
            ),
            nn.BatchNorm2d(hidden_channels),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=hidden_channels,
                out_channels=hidden_channels,
                kernel_size=3,
                padding=1
            ),
            nn.BatchNorm2d(hidden_channels),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=hidden_channels,
                out_channels=hidden_channels,
                kernel_size=3,
                padding=1
            ),
            nn.BatchNorm2d(hidden_channels),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=hidden_channels,
                out_channels=out_channels,
                kernel_size=1
            )
        )

    def forward(self, x):
        out = self.block(x) + self.identity(x)
        return F.leaky_relu(out)


In [5]:
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        blocks = []
        curr_channel = config.in_channels
        for out_channel in config.channel_dims:
            blocks.append(
                nn.Sequential(
                    # reduce spatial dims increase channels
                    nn.Conv2d(
                        in_channels=curr_channel,
                        out_channels=out_channel,
                        kernel_size=3,
                        stride=2,
                        padding=1
                    ),
                    nn.BatchNorm2d(out_channel),
                    nn.LeakyReLU(),
                    # continue at current dim and channels
                    nn.Sequential(*[
                        ResBlock(
                            in_channels=out_channel,
                            out_channels=out_channel
                        ) for _ in range(config.nblocks)
                    ])
                ),
            )
            curr_channel = out_channel

        self.blocks    = nn.Sequential(*blocks)
        self.fc_mu     = nn.Linear(config.channel_dims[-1]*4, config.latent_dim)
        self.fc_logvar = nn.Linear(config.channel_dims[-1]*4, config.latent_dim)

    def forward(self, x):
        x = self.blocks(x)
        x = torch.flatten(x, start_dim=1)
        return self.fc_mu(x), self.fc_logvar(x)

In [6]:
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.latent_map = nn.Linear(config.latent_dim, config.channel_dims[-1]*4)

        blocks = []
        reverse_channels = config.channel_dims
        reverse_channels = list(config.channel_dims)[::-1]
        for i in range(len(reverse_channels)-1):
            blocks.append(
                nn.Sequential(
                    # continue at current dim and channels
                    nn.Sequential(*[
                        ResBlock(
                            in_channels=reverse_channels[i],
                            out_channels=reverse_channels[i]
                        ) for _ in range(config.nblocks)
                    ]),
                    # reduce channels increase spatial dims
                    nn.ConvTranspose2d(
                        in_channels=reverse_channels[i],
                        out_channels=reverse_channels[i+1],
                        kernel_size=3,
                        stride = 2,
                        padding=1,
                        output_padding=1
                    ),
                    nn.BatchNorm2d(reverse_channels[i+1]),
                    nn.LeakyReLU()
                )
            )
        self.blocks = nn.Sequential(*blocks)
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=reverse_channels[-1],
                out_channels=config.in_channels,
                kernel_size=3,
                stride = 2,
                padding=1,
                output_padding=1
            ),
            nn.LeakyReLU(),
            nn.Tanh()
        )

        self.config = config

    def forward(self, z):
        z = self.latent_map(z)
        z = z.view(-1, self.config.channel_dims[-1], 2, 2)
        z = self.blocks(z)
        return self.final_layer(z)


In [7]:
class VAE(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
        self.config = config

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def loss(self, x_hat, x, mu, logvar):
        MSE = F.mse_loss(x_hat, x)
        MKLD = torch.mean(-0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1), dim=0)
        loss = MSE + self.config.kld_weight * MKLD
        return loss, MSE, MKLD

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar

In [13]:
class LitVAE(pl.LightningModule):
    def __init__(self, model, config=None):
        super().__init__()
        self.model = model
        self.config = config
        self.lr = config.lr

        if self.logger:
            self.logger.experiment.config.update(config)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_hat, mu, logvar = self(x)

        loss, MSE, MKLD = self.model.loss(x_hat, x, mu, logvar)
        self.log('train/loss', loss, prog_bar=True)
        self.log('train/MSE', MSE, prog_bar=True)
        self.log('train/MKLD', MKLD, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x_hat, mu, logvar = self(x)

        loss, MSE, MKLD = self.model.loss(x_hat, x, mu, logvar)
        self.log('val/loss', loss, prog_bar=True)
        self.log('val/MSE', MSE, prog_bar=True)
        self.log('val/MKLD', MKLD, prog_bar=True)

        if batch_idx == 0:
            n_images = min(x.size(0), 8)
            comparison = torch.cat([x[:n_images], x_hat[:n_images]])
            grid = torchvision.utils.make_grid(comparison)
            self.logger.experiment.log({"val/reconstructions": [wandb.Image(grid, caption="Top: Original, Bottom: Reconstructed")]})

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, betas=(self.config.beta1, self.config.beta2))
        if self.config.use_lr_schedule:
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.config.max_epochs)
            return [optimizer], [scheduler]
        return optimizer


# Config

In [21]:
def get_num_downsample_layers(img_size):
    """
    Get the number of strided Conv2D layers
    required to produce a 2x2 output volume
    """
    if img_size < 2:
        raise ValueError("Image size must be at least 2x2.")

    # Calculate the minimum number of downsample layers required for 2x2 final
    num_layers = math.ceil(math.log2(img_size / 2))
    return num_layers

def build_channel_dims(start_channels, nlayers):
    """
    Construct a list of channel counts for nlayers downsample layers
    assuming the channels double as spatial dims halve
    """
    channels = []
    for _ in range(nlayers):
        channels.append(start_channels)
        start_channels *= 2
    return channels

class CIFAR10VAEConfig:
    def __init__(self):
        self.checkpoint_path = "./checkpoints"
        self.save_top_k = 1
        self.batch_size = 2048 # TODO: maxout for max throughput
        self.max_epochs = 60
        self.use_lr_schedule = False
        self.lr = 3e-4
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.img_size = 32
        self.in_channels = 3
        self.latent_dim = 128
        self.kld_weight = 0.0000025
        self.start_channels = 32
        self.nblocks = 1
        self.nlayers = get_num_downsample_layers(self.img_size)
        self.channel_dims = build_channel_dims(self.start_channels, self.nlayers)

    def update(self, updates):
        for key, value in updates.items():
            if hasattr(self, key):
                setattr(self, key, value)
        self.nlayers = get_num_downsample_layers(self.img_size)
        self.channel_dims = build_channel_dims(self.start_channels, self.nlayers)

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items()}

# Train

In [22]:
config = CIFAR10VAEConfig()
model = VAE(config)
lit_model = LitVAE(model, config)
cifar10_data = CIFAR10DataModule(config)

wandb.init(project="VAE CIFAR-10", config=config.to_dict())
wandb_logger = WandbLogger(project="VAE CIFAR-10", log_model=False)
wandb_logger.watch(lit_model, log="all")

lr_monitor = LearningRateMonitor(logging_interval='step')

checkpoint_callback = ModelCheckpoint(
    dirpath=config.checkpoint_path,
    filename='model-{epoch:02d}-{val_loss:.2f}',
    every_n_epochs=5,
    save_top_k=config.save_top_k,
    monitor='val/loss',
    mode='min',
    save_last=True
)

# Define the EarlyStopping callback
early_stop_callback = EarlyStopping(
    monitor='val/loss',
    min_delta=0.00,
    patience=3,
    verbose=True,
    check_finite=True
)

In [ ]:
#wandb.finish()

In [ ]:
trainer = pl.Trainer(
    max_epochs=config.max_epochs,
    devices=1,
    accelerator="gpu",
    precision="16-mixed",
    logger=wandb_logger,
    callbacks=[
        lr_monitor,
        early_stop_callback,
        # checkpoint_callback
    ],
    log_every_n_steps=1,
)

# tuner = Tuner(trainer)
# tuner.lr_find(lit_model, datamodule=cifar10_data)

In [ ]:
trainer.fit(lit_model, cifar10_data)
wandb.finish()

# Sweeps

In [ ]:
default_sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val/loss',
        'goal': 'minimize'
    },
    'parameters': {
        'nblocks': {
            'values': [0, 1, 2]
        },
        'latent_dim': {
            'values': [128, 256, 512]
        },
        'start_channels': {
            'values': [32, 64]
        },
        'lr': {
            'min': 1e-5,
            'max': 1e-2,
            'distribution': 'log_uniform_values'
        },
        'kld_weight': {
            'min': 0.0000025,
            'max': 1,
            'distribution': 'log_uniform_values'
        }
    }
}

def build_sweeps_for_param(default_config, param_name, param_values):
    '''
    Builds sweep configs for each value of the scientific parameter.
    '''
    sweep_configs = []
    for val in param_values:
        config = deepcopy(default_config)
        config['name'] = f'{param_name}-{val}-sweep'
        config['parameters'][param_name] = {'values': [val]}
        sweep_configs.append(config)
    return sweep_configs

def get_sweep_ids_for_configs(sweep_configs, project_name):
    sweep_ids = []
    for config in sweep_configs:
        sweep_id = wandb.sweep(config, project=project_name)
        sweep_ids.append(sweep_id)

    return sweep_ids

def run_sweeps(sweep_ids, train_func, count=15):
    for sweep_id in sweep_ids:
        wandb.agent(sweep_id, train_func, count=count)

In [ ]:
def train():
    with wandb.init() as run:
        config = CIFAR10VAEConfig()
        config.update(wandb.config)

        model = VAE(config)
        lit_model = LitVAE(model, config)
        cifar10_data = CIFAR10DataModule(config)

        wandb_logger = WandbLogger(project="VAE CIFAR-10", log_model=False)

        early_stop_callback = EarlyStopping(
            monitor='val/loss',
            min_delta=0.00,
            patience=3,
            verbose=True,
            check_finite=True
        )

        trainer = pl.Trainer(
            max_epochs=config.max_epochs,
            devices=1,
            accelerator="gpu",
            precision="16-mixed",
            logger=wandb_logger,
            callbacks=[early_stop_callback]
        )

        trainer.fit(lit_model, cifar10_data)
    wandb.finish()

In [ ]:
nblocks_sweep_configs = build_sweeps_for_param(default_sweep_config, 'nblocks', [0, 1, 2])
latent_dim_sweep_configs = build_sweeps_for_param(default_sweep_config, 'latent_dim', [128, 256, 512])
# kld_weight_sweep_configs = build_sweeps_for_param(default_sweep_config, 'kld_weight', []) # TODO: either define discrete set or specify range if able

In [ ]:
nblocks_sweep_ids = get_sweep_ids_for_configs(nblocks_sweep_configs, project_name='VAE CIFAR-10')

run_sweeps(nblocks_sweep_ids, train)